# Recognising faces using some classification algorithms

## logistic, KNN, SVM etc.
    1.  load the training data (numpy arrays of all persons)
        x : values are stored in numpy arrays
        y : values we need to assign for each person
    2. Read a video stream using opencv
    3. extract faces out of it
    4. use knn to find the prediction of face (int)
    5. map the predicted id to name of the user
    6. display the predictions on screen - bounding box and name

In [8]:
import cv2
import numpy as np
import os

In [9]:
# returns the euclidean distance ie underroot of sum of squares of (x1-x2)
def distance(v1, v2):
	# Eucledian 
	return np.sqrt(((v1-v2)**2).sum())

def knn(train, test, k=5):
	dist = []
	
	for i in range(train.shape[0]):
		# Get the vector and label
		ix = train[i, :-1]
		iy = train[i, -1]
		# Compute the distance from test point
		d = distance(test, ix)
		dist.append([d, iy])
	# Sort based on distance and get top k
	dk = sorted(dist, key=lambda x: x[0])[:k]
	# Retrieve only the labels
	labels = np.array(dk)[:, -1]
	
	# Get frequencies of each label
	output = np.unique(labels, return_counts=True)
	# Find max frequency and corresponding label
	index = np.argmax(output[1])
	return output[0][index]

In [10]:
#Init Camera
cap = cv2.VideoCapture(0)

# Face Detection Classifier
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip = 0
dataset_path = './data/'
face_data = []
labels = []

# labels for given file
class_id = 0

# used to create mapping from id to user's name
names = {}

### Data Preparation

In [11]:
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        
        # Create mapping between class_id ie the label and name
        names[class_id] = fx[:-4]
        print("Loaded: " + fx)
        
        # we load the file
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        # Create labels for class
        # create array of ones with nub=mber of rows = number of faces captured in that .npy file
        target = class_id * np.ones((data_item.shape[0]))
        
        # increment class_id by one for next .npy file
        class_id = class_id + 1
        labels.append(target)

Loaded: vinayakk.npy


In [12]:
# X_train
face_dataset = np.concatenate(face_data, axis=0)

# Y_train
face_labels = np.concatenate(labels, axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

(88, 30000)
(88, 1)


In [13]:
# Concatenate X and Y both into 1 training matrix
# train_set = [[X][Y]]

# 30,000 features + 1 label
train_set = np.concatenate((face_dataset, face_labels),axis = 1)
print(train_set.shape)

(88, 30001)


### Reading test data using video stream

In [14]:
while(True):
    ret, frame = cap.read()
    if(ret==False):
        continue
        
    faces = face_cascade.detectMultiScale(frame, 1.3,5)
    
    for face in faces:
        x, y, w, h = face
        
        # Get the REGION OF INTEREST
        offset = 10
        face_section = frame[y-offset: y+h+offset, x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))

        # Predicted label(out)
        out = knn(train_set, face_section.flatten())
        
        # Display on screen the name and a rectangle around it
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame, (x,y),(x+w,y+h),(0,255,255),2)
    
    cv2.imshow("Faces", frame)
    
    key = cv2.waitKey(1) & 0xFF
    if(key==ord('q')):
        break

cap.release()
cv2.destroyAllWindows()